# Checking jpn and cmn languages

In [58]:
import pickle
import numpy as np
from sklearn.preprocessing import normalize
from scipy import spatial
import json

Data format:  
{lang : swad_list, raw_emb_list, emb_fn, not_found_list}

In [2]:
load_fn = '/mnt/store/eszti/projects/dipterv/univ_embedding/output_permanent/20170422_2020_27/get_embed_proc/get_embed_proc.pickle'
with open(load_fn) as f:
    data = pickle.load(f)

eng = data['eng']
jpn = data['jpn']
cmn = data['cmn']

Checking missing swadesh entries:

In [18]:
print('Eng: {}'.format(eng[3]))
print('Jpn: {}'.format(jpn[3]))
print('Cmn: {}'.format(cmn[3]))

Eng: []
Jpn: [11, 25, 62, 78, 94]
Cmn: [7, 12, 31, 82]


In [28]:
def filter(emb, idxs):
    fil, idx_map = [], []
    for i, e in enumerate(emb):
        if i in idxs:
            continue
        else:
            fil.append(e)
            idx_map.append(i)
    return fil, idx_map
        

eng_emb_fil, eng_map = filter(eng[1], eng[3])
jpn_emb_fil, jpn_map = filter(jpn[1], jpn[3])
cmn_emb_fil, cmn_map = filter(cmn[1], cmn[3])

eng_emb = normalize(np.array(eng_emb_fil).astype(np.float32))
jpn_emb = normalize(np.array(jpn_emb_fil).astype(np.float32))
cmn_emb = normalize(np.array(cmn_emb_fil).astype(np.float32))

In [50]:
def get_cos_sim_mx(emb):
    mx = 1 - spatial.distance.cdist(emb, emb, 'cosine')
    return mx

eng_cos_mx = get_cos_sim_mx(eng_emb)
jpn_cos_mx = get_cos_sim_mx(jpn_emb)
cmn_cos_mx = get_cos_sim_mx(cmn_emb)

In [51]:
def argsort_mx(emb):
    return [row.argsort()[::-1] for row in emb]

eng_swad = eng[0]

def get_similars_dict(argsort_mx, idx_map):
    ret = dict()
    i = 0
    for row in argsort_mx:
        ret[eng_swad[idx_map[i]]] = [eng_swad[idx_map[idx]] for idx in row]
        i += 1
    return ret    

In [54]:
eng_sort = argsort_mx(eng_cos_mx)
jpn_sort = argsort_mx(jpn_cos_mx)
cmn_sort = argsort_mx(cmn_cos_mx)

In [55]:
eng_sim_dict = get_similars_dict(eng_sort, eng_map)
jpn_sim_dict = get_similars_dict(jpn_sort, eng_map)
cmn_sim_dict = get_similars_dict(cmn_sort, eng_map)

In [60]:
eng_fn = '/home/eszti/data/misc/check_jpn_cmn/eng.json'
jpn_fn = '/home/eszti/data/misc/check_jpn_cmn/jpn.json'
cmn_fn = '/home/eszti/data/misc/check_jpn_cmn/cmn.json'

def save_as_json(d, fn):
    with open(fn, 'w') as f:
        json.dump(d, f)
        
save_as_json(eng_sim_dict, eng_fn)
save_as_json(jpn_sim_dict, jpn_fn)
save_as_json(cmn_sim_dict, cmn_fn)

Manual testing:

In [63]:
print('Eng: {}\n'.format(eng_sim_dict['dog']))
print('Jpn: {}\n'.format(jpn_sim_dict['dog']))
print('Cmn: {}\n'.format(cmn_sim_dict['dog']))

Eng: [u'dog', u'bite', u'louse', u'claw', u'bird', u'snake', u'eat', u'nose', u'hair', u'ear', u'fat', u'tooth', u'bone', u'fish', u'foot', u'tail', u'egg', u'cold', u'earthworm', u'hot', u'eye', u'feather', u'tongue', u'bark', u'hide', u'mouth', u'flesh', u'little', u'drink', u'kill', u'stand', u'sleep', u'go', u'night', u'head', u'neck', u'big', u'hand', u'good', u'dead', u'blood', u'heart', u'horn', u'swim', u'liver', u'human', u'hear', u'moon', u'say', u'grind', u'sit', u'that', u'fire', u'rock', u'black', u'know', u'tree', u'white', u'what', u'fly', u'come', u'red', u'breast', u'sand', u'we', u'thou', u'yellow', u'short', u'adulterous', u'smoke', u'dry', u'name', u'male', u'long', u'not', u'hill', u'knee', u'rain', u'root', u'close', u'star', u'thin', u'path', u'who', u'female', u'abdomen', u'wind', u'leaf', u'many', u'water', u'salt', u'this', u'dull', u'year', u'all', u'lie', u'cloud', u'sun', u'earth', u'see', u'i', u'burn', u'give', u'round', u'distant', u'ash', u'seed', u'gre